In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
import os 
import numpy as np

In [52]:
#Convolutional encoder for one dimensional input data, the output is also one dimensional and should look like 
#the input data but with a smaller size. It is used to compress the data and extract features from it.


class ConvEncoder(nn.Module):

    def __init__(self, input_size, output_size, kernel_size, stride, padding):
        super(ConvEncoder, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.conv1 = nn.Conv1d(1, 1, kernel_size, stride, padding)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(1, 1, kernel_size, stride, padding)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)
        #Calculate size of linear layer input
        size = (input_size - kernel_size + 2*padding)/stride + 1
        size = size /2/2
        size = int(size)
        self.linear = nn.Linear(size, output_size)

    def forward(self, x):
        x = self.conv1(x)
        print("After conv1: ", x.size())
        x = self.relu1(x)
        print("After relu1: ", x.size())
        x = self.pool1(x)
        print("After pool1: ", x.size())
        x = self.conv2(x)
        print("After conv2: ", x.size())
        x = self.relu2(x)
        print("After relu2: ", x.size())
        x = self.pool2(x)
        print("After pool2: ", x.size())
        x = self.linear(x)
        print("After linear: ", x.size())
        return x
#Test it
random_input = torch.randn(1, 1, 50)
print(random_input.size())
encoder = ConvEncoder(random_input.shape[-1], 5, 3, 1, 1)
output = encoder(random_input)
print(output)
print(output.size())



torch.Size([1, 1, 50])
After conv1:  torch.Size([1, 1, 50])
After relu1:  torch.Size([1, 1, 50])
After pool1:  torch.Size([1, 1, 25])
After conv2:  torch.Size([1, 1, 25])
After relu2:  torch.Size([1, 1, 25])
After pool2:  torch.Size([1, 1, 12])
After linear:  torch.Size([1, 1, 5])
tensor([[[-0.6439,  0.2825,  0.2586, -0.9399,  0.0994]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 1, 5])


In [55]:
#Decoder

class ConvDecoder(nn.Module):
    
        def __init__(self, input_size, output_size, kernel_size, stride, padding):
            super(ConvDecoder, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            self.kernel_size = kernel_size
            self.stride = stride
            self.padding = padding
            self.deconv1 = nn.ConvTranspose1d(1, 1, kernel_size, stride, padding)
            self.relu1 = nn.ReLU()
            self.deconv2 = nn.ConvTranspose1d(1, 1, kernel_size, stride, padding)
            self.relu2 = nn.ReLU()
            self.upsample1 = nn.Upsample(scale_factor=2, mode='nearest')
            self.upsample2 = nn.Upsample(scale_factor=2, mode='nearest')
            size = 
            self.linear = nn.Linear(input_size, output_size)

    
        def forward(self, x):
            x = self.deconv1(x)
            print("After deconv1: ", x.size())
            x = self.relu1(x)
            print("After relu1: ", x.size())
            x = self.upsample1(x)
            print("After upsample1: ", x.size())
            x = self.deconv2(x)
            print("After deconv2: ", x.size())
            x = self.relu2(x)
            print("After relu2: ", x.size())
            x = self.upsample2(x)
            print("After upsample2: ", x.size())
            return x
        
#Test it
random_input = torch.randn(1,1,5)
print(random_input.size())
decoder = ConvDecoder(random_input.shape[-1], 50, 3, 1, 1)
output = decoder(random_input)
print(output)

torch.Size([1, 1, 5])
After deconv1:  torch.Size([1, 1, 5])
After relu1:  torch.Size([1, 1, 5])
After upsample1:  torch.Size([1, 1, 10])
After deconv2:  torch.Size([1, 1, 10])
After relu2:  torch.Size([1, 1, 10])
After upsample2:  torch.Size([1, 1, 20])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<UpsampleNearest1DBackward0>)
